In [62]:
import os
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join

import time
import datetime as dt
from datetime import timedelta, datetime, timezone
from dateutil.relativedelta import relativedelta
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False

In [63]:
# Plot 설정

GRAPH_TITLE = "4월 {sensor} 교통량"
X_AXIS_LABEL = "시간"
Y_AXIS_LABEL = "교통량"
LEGEND_2023 = "2023/04/06"
LEGEND_2024 = "2023/04/04"
Month = "4월"
Data_Type = "교통량"
X_TICK_INTERVAL = 60  # Adjust interval for x-axis tick labels (e.g., every 60 entries)
OUTPUT_DIRECTORY = 'VDS_plot_results'  # Directory to save plots
# 결과 폴더 생성
directionfolder = ['도솔대교_VDS_EtoW', '도솔대교_VDS_WtoE']
if not os.path.exists(OUTPUT_DIRECTORY):
    os.makedirs(OUTPUT_DIRECTORY)
    for d in directionfolder:
        os.makedirs(OUTPUT_DIRECTORY + "/" + d)

# vds 데이터 폴더, 파일
data_folder_name = "vds_data"
data_folder_year_date = ['202304', '202404']

# 데이터 폴더 존재하지 않을 시 데이터 폴더 생성, 해당 폴더 안에 데이터 파일을 넣어야함
if not os.path.exists("vds_data"):
    os.makedirs("vds_data")
    os.makedirs("vds_data/202304")
    os.makedirs("vds_data/202404")

In [64]:
# 데이터 폴더 내 파일명 확인
filenames = []
filenames_2023 = []
filenames_2024 = []
for i in data_folder_year_date:
    path = data_folder_name + "/" + i
    for f in listdir(path):
        if(isfile(join(path, f))):
            filenames.append(data_folder_name + "/" + i + "/" + f)
            if('2023' in i and '_new' not in f):
                filenames_2023.append(data_folder_name + "/" + i + "/" + f)
            if('2024' in i and '_new' not in f):
                filenames_2024.append(data_folder_name + "/" + i + "/" + f)
                
# 정의
directionstr = ['EtoW', 'WtoE']
EtoW = ['VDS0022', 'VDS0103', 'VDS0109', 'VDS0123']
WtoE = ['VDS0023', 'VDS0104', 'VDS0110', 'VDS0124']
directionrev = [WtoE, EtoW]
peak = ['오전첨두', '비첨두', '오후첨두']
times = [['07:00~08:00', '08:00~09:00', '07:00~09:00'],
         ['12:00~13:00', '13:00~14:00', '12:00~14:00'],
         ['17:00~18:00', '18:00~19:00', '17:00~19:00']]
months = ['2월', '3월', '4월']

In [65]:
#VDS ID 

vdsIDs = []
vds_coord = '도솔대교_VDS_좌표.csv'
df = pd.read_csv(vds_coord)

vds_id = '도솔대교_VDS_ID.csv'
ids = pd.read_csv(vds_id)
for index, row in df.iterrows():
    vdsIDs.append(row['vds_id'])

In [66]:
# new 2023 files preprocessing

vds_link = pd.read_csv('[MERGED]total_VDS_with_osm.csv', encoding='cp949')
vds_sensors = ['VDS0022', 'VDS0023', 'VDS0103', 'VDS0104', 'VDS0107', 'VDS0108', 'VDS0109', 'VDS0110', 
               'VDS0112', 'VDS0113', 'VDS0123',  'VDS0124']

data_to_use = ['vds_tr_vol_5mn', 'vds_trvl_spd_5mn', 'vds_trvl_tm_5mn']

for i in filenames_2023:
    result = pd.DataFrame(columns = ['reg_ymdhms', 'vds_id', 'tr_vol', 'trvl_spd', 'trvl_tm'])
    
    vds_data = pd.read_csv(i)

    for sensor in vds_sensors:
        LINK_ID_da = vds_link.loc[vds_link['VDS_ID'] == sensor]['LINK_ID_da'].iloc[0]
        LINK_ID = vds_link.loc[vds_link['VDS_ID'] == sensor]['LINK_ID'].iloc[0]
    
        sensordata = vds_data.loc[(vds_data['link_id'] == LINK_ID) & (vds_data['road_route_id'])]
    
        for index, row in sensordata.iterrows():
            data = [{
                'reg_ymdhms' : str(row['ymd_id']) + str(row['hh_id']).zfill(2) + str(row['mn_id']).zfill(2) + '00',
                'vds_id' : sensor,
                'tr_vol' : row['vds_tr_vol_5mn'], 'trvl_spd' : row['vds_trvl_spd_5mn'],
                'trvl_tm' : row['vds_trvl_tm_5mn']
            }]
    
            resapp = pd.DataFrame(data)
            result = pd.concat([result, resapp], ignore_index=True)

    new_filename = i.replace('.csv', '_new.csv')
    result.to_csv(new_filename, index=False, encoding='utf-8-sig')

In [67]:
# Load the data
file_2023 = filenames_2023[0].replace('.csv', '_new.csv')
file_2024 = filenames_2024[0]

data_2023 = pd.read_csv(file_2023)
data_2024 = pd.read_csv(file_2024)

# Convert timestamps to datetime for easier manipulation
data_2023['reg_ymdhms'] = pd.to_datetime(data_2023['reg_ymdhms'], format='%Y%m%d%H%M%S')
data_2024['reg_ymdhms'] = pd.to_datetime(data_2024['reg_ymdhms'], format='%Y%m%d%H%M%S')

# Resample 2024 data to 5-minute intervals to match 2023
data_2024_resampled = (
    data_2024.groupby(['vds_id', pd.Grouper(key='reg_ymdhms', freq='5T')])
    .agg({'tr_vol': 'sum'})
    .reset_index()
)

# Combine datasets for plotting
data_combined = pd.concat([
    data_2023[['reg_ymdhms', 'vds_id', 'tr_vol']],
    data_2024_resampled
])

# Extract time of day explicitly for plotting
data_combined['time_of_day'] = data_combined['reg_ymdhms'].dt.strftime('%H:%M:%S')
data_combined['year_month_day'] = data_combined['reg_ymdhms'].dt.strftime('%Y-%m-%d')

# Separate data for each year
data_2023_plot = data_combined[data_combined['reg_ymdhms'].dt.year == 2023]
data_2024_plot = data_combined[data_combined['reg_ymdhms'].dt.year == 2024]

# Create output directory if it doesn't exist
os.makedirs(OUTPUT_DIRECTORY, exist_ok=True)

# Extract unique sensors
unique_sensors = data_combined['vds_id'].unique()

# Generate plots for each sensor
for sensor in unique_sensors:
    # Data for the specific sensor
    sensor_data_2023 = data_2023_plot[data_2023_plot['vds_id'] == sensor]
    sensor_data_2024 = data_2024_plot[data_2024_plot['vds_id'] == sensor]

    # Generate the plot
    plt.figure(figsize=(12, 6))

    # Plot 2023 data
    plt.plot(sensor_data_2023['time_of_day'], sensor_data_2023['tr_vol'], label=LEGEND_2023, linewidth=1.5)

    # Plot 2024 data
    plt.plot(sensor_data_2024['time_of_day'], sensor_data_2024['tr_vol'], label=LEGEND_2024, linewidth=1.5)

    # Adjust x-axis ticks for better clarity
    plt.xticks(sensor_data_2023['time_of_day'][::X_TICK_INTERVAL], rotation=45)

    plt.title(GRAPH_TITLE.format(sensor=sensor))
    plt.xlabel(X_AXIS_LABEL)
    plt.ylabel(Y_AXIS_LABEL)
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.legend()

    # Save the plot
    if (sensor in EtoW):
        output_file = OUTPUT_DIRECTORY + '/도솔대교_VDS_EtoW/' + Month + "_" +  sensor +"_" +  Data_Type + ".png"
        plt.savefig(output_file)
        plt.close()
    elif (sensor in WtoE):
        output_file = OUTPUT_DIRECTORY + '/도솔대교_VDS_WtoE/' + Month + "_" +  sensor +"_" +  Data_Type + ".png"
        plt.savefig(output_file)
        plt.close()        
    else:
        output_file = OUTPUT_DIRECTORY + '/도솔대교_VDS_EtoW/' + Month + "_" +  sensor +"_" +  Data_Type + ".png"
        plt.savefig(output_file)
        output_file = OUTPUT_DIRECTORY + '/도솔대교_VDS_WtoE/' + Month + "_" +  sensor +"_" +  Data_Type + ".png"
        plt.savefig(output_file)
        plt.close()